In [3]:
import pandas as pd
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [1]:
from gensim.models import Word2Vec

In [2]:
pandarallel.initialize(progress_bar=True)

NameError: name 'pandarallel' is not defined

In [3]:
dataRew=pd.read_csv('../Dataset/datiClean.csv')
dataMovie=pd.read_csv('../Dataset/movieclean.csv')

In [4]:
dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

In [5]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(ast.literal_eval)

In [6]:
dataRew.drop(['review_text'],axis=1,inplace=True)

In [7]:
dataMovie.drop(['plot_synopsis','plot_summary'],axis=1,inplace=True)

### Split the Dataset

important distribute well the labels in the train and test dataset

In [8]:
dataRew.drop(['review_date','movie_id','user_id','rating','review_summary'],axis=1,inplace=True)

In [9]:
x=dataRew['clean_review']
y=dataRew['is_spoiler']

In [10]:
## Stratify bilancia i dataset 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)

In [11]:
y_train.value_counts()

is_spoiler
False    338391
True     120739
Name: count, dtype: int64

In [12]:
y_test.value_counts()

is_spoiler
False    84598
True     30185
Name: count, dtype: int64

### Apply Logistic Regression and Naive Bayes, with k fold

### Function for computing the result of the model

In [89]:
def print_mean():
    # Calcola le medie delle metriche
    mean_accuracy = np.mean(metrics['accuracy'])
    mean_precision = np.mean(metrics['precision'])
    mean_recall = np.mean(metrics['recall'])
    mean_f1_score = np.mean(metrics['f1_score'])

    # Stampa le medie delle metriche
    print("Mean Accuracy:", mean_accuracy)
    print("Mean Precision:", mean_precision)
    print("Mean Recall:", mean_recall)
    print("Mean F1 Score:", mean_f1_score)

In [86]:
def print_test(y_pred_test):
    # Calcolo delle metriche di valutazione sul set di test
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)

    # Stampa delle metriche di valutazione sul set di test
    print("Test Accuracy:", accuracy_test)
    print("Test Precision:", precision_test)
    print("Test Recall:", recall_test)
    print("Test F1 Score:", f1_score_test)

In [91]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def computeLogistic(folds,iter,X,y_train):
    logistic_reg=LogisticRegression(max_iter=iter)
    ## Stratified k-fold grant me a balance division of classes

    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        logistic_reg.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = logistic_reg.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return logistic_reg

In [90]:
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}
def compute_naive(folds,X,y_train):
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    naive_bayes = MultinomialNB()
    
    for train_index, val_index in kf.split(X, y_train):
        X_fold_train, X_fold_val = X[train_index], X[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Training
        naive_bayes.fit(X_fold_train, y_fold_train)
        
        # Validation
        y_pred = naive_bayes.predict(X_fold_val)
        
        # Compute metrics
        metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        metrics['precision'].append(precision_score(y_fold_val, y_pred))
        metrics['recall'].append(recall_score(y_fold_val, y_pred))
        metrics['f1_score'].append(f1_score(y_fold_val, y_pred))
    return naive_bayes

### Using Bag of Words

first create the text, using the array of vector

In [16]:
text=[" ".join(word) for word in X_train]

In [55]:
textT=[" ".join(word) for word in X_test]

In [75]:
## Bag of Words for train
vect=CountVectorizer()
X=vect.fit_transform(text)

In [76]:
## Bag of Words for test
X_t=vect.transform(textT)

In [77]:
y_train=y_train.values
y_test=y_test.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

## Logistic Regression

### Result Train and Validation

In [47]:
logistic_reg=computeLogistic(5,1000,X,y_train)
print_mean()

Mean Accuracy: 0.7682813146603359
Mean Precision: 0.6005845863071715
Mean Recall: 0.35495162254321866
Mean F1 Score: 0.44617830623862076


### Result Test

In [58]:
y_pred_test = logistic_reg.predict(X_t)
print_test(y_pred_test)


Test Accuracy: 0.7682322295113387
Test Precision: 0.5999665103817816
Test Recall: 0.35610402517806855
Test F1 Score: 0.4469345751647575


## Naive Bayes

### Result Validation, Training

In [82]:
naive_bayes=compute_naive(5,X,y_train)
print_mean()

Mean Accuracy: 0.7379326116786096
Mean Precision: 0.5017765946467418
Mean Recall: 0.48879815549723704
Mean F1 Score: 0.49519856266242845


### Result Test

In [83]:
y_pred_test = naive_bayes.predict(X_t)

print_test(y_pred_test)

Test Accuracy: 0.7393080856921321
Test Precision: 0.50453789663295
Test Recall: 0.4825244326652311
Test F1 Score: 0.4932856925135048


## Tf-idf

In [92]:

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(text)
X_test_tfidf = tfidf_vectorizer.transform(textT)

### Logistic Regression

In [ ]:
logistic_reg=computeLogistic(5,1000,X_train_tfidf,y_train)
print_mean()

### Naive Bayes

In [93]:
naive_bayes=compute_naive(5,X_train_tfidf,y_train)
print_mean()

Mean Accuracy: 0.740912159954697
Mean Precision: 0.7029707118055426
Mean Recall: 0.025600675433397252
Mean F1 Score: 0.04940127551786295


### Word2Vec

ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Python311\Lib\site-packages\scipy\linalg\__init__.py)

### Apply Pre-Trained Model

In [ ]:
text